# `Практикум по программированию на языке Python`
<br>

## `Numpy и матрично-векторные вычисления`

## `Роман Ищенко (roman.ischenko@gmail.com)`

#### `Москва, 2023`

О чём можно узнать из этого ноутбука:

* операции при работе с массивами
* многомерные массивы
* изменение размеров массивов
* broadcasting
* продвинутая индексация
* view и копирование
* свёртка
* разные прикладные задачи

In [1]:
import warnings
warnings.filterwarnings('ignore')

### `Представление матрицы в Python`

Простейший вариант - список списков:

In [2]:
A = [
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 9],
]

B = [
    [1, 0, 0],
    [0, 2, 0],
    [0, 0, 3],
]

def print_matrix(A):
    for row in A:
        print(row)
    print()

print_matrix(A)
print_matrix(B)

[1, 2, 3]
[4, 5, 6]
[7, 8, 9]

[1, 0, 0]
[0, 2, 0]
[0, 0, 3]



### `Представление матрицы в Python`

<font color='brown'>**Задача 1.** Задайте матрицу `C` размером $9$ на $10$, в которой $ij$-ый элемент будет равен $10 i + j$. Не используйте библиотеку `numpy`!</font>

In [3]:
### your code here
C = []
for idx in range(9):
    C.append([])
    for jdx in range(10):
        C[-1].append(10 * idx + jdx)

Проверьте себя:

In [4]:
assert len(C) == 9
assert len(C[0]) == 10
assert C[8][3] == 83

print_matrix(C)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
[20, 21, 22, 23, 24, 25, 26, 27, 28, 29]
[30, 31, 32, 33, 34, 35, 36, 37, 38, 39]
[40, 41, 42, 43, 44, 45, 46, 47, 48, 49]
[50, 51, 52, 53, 54, 55, 56, 57, 58, 59]
[60, 61, 62, 63, 64, 65, 66, 67, 68, 69]
[70, 71, 72, 73, 74, 75, 76, 77, 78, 79]
[80, 81, 82, 83, 84, 85, 86, 87, 88, 89]



### `Представление матрицы в Python`

Одна из проблем списков — отсутствие поэлементных и матричных операций.

Другой недостаток — работа со списками не позволяет использовать векторные инструкции в процессоре, которые на порядки ускоряют матричные вычисления

Попробуем самостоятельно реализовать несколько методов для матричных вычислений, работающих с представлением матриц в виде списков.

Для оценки производительности будем использовать *декооратор* `timed`:

In [5]:
def timed(method):
    import time
    def __timed(*args, **kw):
        time_start = time.time()
        result = method(*args, **kw)
        time_end = time.time()

        print('{}  {:.3f} ms\n'.format(method.__name__,
                                      (time_end - time_start) * 1000))
        return result

    return __timed

### `Опишем базовые операции: сложение`

In [7]:
@timed
def matrix_add(A, B):
    # skip correctness of dimensions check

    n, m = len(A), len(A[0])
    C = [[0.0 for _ in range(m)] for _ in range(n)]

    for i in range(n):
        for j in range(m):
            C[i][j] = A[i][j] + B[i][j]
    return C

print_matrix(matrix_add(A, B))

matrix_add  0.013 ms

[2, 2, 3]
[4, 7, 6]
[7, 8, 12]



### `Опишем базовые операции: транспонирование`

In [8]:
@timed
def matrix_transpose(A):
    n, m = len(A), len(A[0])
    C = [[0.0 for _ in range(n)] for _ in range(m)]

    for i in range(n):
        for j in range(m):
            C[j][i] = A[i][j]
    return C

print_matrix(matrix_transpose(A))

matrix_transpose  0.012 ms

[1, 4, 7]
[2, 5, 8]
[3, 6, 9]



### `Опишем базовые операции: умножение`

In [9]:
def dot(a, b):
    return sum(x * y for (x, y) in zip(a, b))

@timed
def matrix_mul(A, B):
    # check correctness of dimensions
    n, m = len(A), len(B[0])
    C = [[0.0 for _ in range(n)] for _ in range(m)]

    B_T = matrix_transpose(B)
    for i in range(n):
        for j in range(m):
            C[i][j] = dot(A[i], B_T[j])
    return C

print_matrix(matrix_mul(A, B))

matrix_transpose  0.009 ms

matrix_mul  1.176 ms

[1, 4, 9]
[4, 10, 18]
[7, 16, 27]



### `Библиотека NumPy`

- Позволяет работать с многомерными массивами<br><br>

- Реализует множество базовых матричных операций<br><br>

- Работает на порядки быстрее самописных реализаций в Python<br><br>

- Поддерживается большинством сторонних модулей, работающих с матрицами<br><br>

In [2]:
# from numpy import * — НЕ ДЕЛАЙТЕ ТАК
import numpy as np

Текущая версия:

In [3]:
np.__version__

'1.23.3'

### `Почему NumPy быстрая`

- Значительная часть кода написана на C<br><br>

- Базовым классом является `ndarray`, имеющий следующие отличия от списков:

    1. NumPy array имеет фиксированную длину, задаваемую в момент его создания (списки в Python могут менять размер динамически)

    2. Все элементы в NumPy array имеют один тип<br><br>

- NumPy array хранится в памяти в виде одного последовательного блока, что позволяет эффективно использовать процессорный кэш и векторные инструкции<br><br>

- NumPy можно подключить к высокооптимизированным библиотекам для матричной алгебры (BLAS, LAPACK, Intel MKL)<br><br>

- Часть матричных операций может быть распараллелена при наличии в системе нескольких потоков


### `Способы создания NumPy array`

1. Пустой
2. Заполненный нулями
3. Заполненный единицами
4. Заполненный нужным значением

In [12]:
print(np.empty(shape=[2, 3]))  # values are arbitrary

[[4.89521082e-310 0.00000000e+000 2.05833592e-312]
 [6.79038654e-313 2.14321575e-312 2.27053550e-312]]


In [13]:
print(np.zeros([2, 3]))

[[0. 0. 0.]
 [0. 0. 0.]]


In [17]:
print(np.ones([2, 3]))

[[1. 1. 1.]
 [1. 1. 1.]]


In [18]:
print(np.full([2, 3], 3.0))

[[3. 3. 3.]
 [3. 3. 3.]]


### `Способы создания NumPy array`

Указанные четыре функции имеют аналоги с суффиксом `*_like`, которые создают массив того же размера, что передан в качестве агрумента:

In [19]:
a = np.zeros([1, 2, 3])
b = np.ones_like(a)

print(a)
print(b)
a.shape, b.shape

[[[0. 0. 0.]
  [0. 0. 0.]]]
[[[1. 1. 1.]
  [1. 1. 1.]]]


((1, 2, 3), (1, 2, 3))

### `Способы создания NumPy array`

Для создания одномерных массивов можно передавать их длину в виде целого числа:  

In [20]:
np.zeros(5)

array([0., 0., 0., 0., 0.])

Для одномерного массива тоже можно писать кортеж:

In [26]:
np.zeros(shape=(5,))

array([0., 0., 0., 0., 0.])

Для больших размерностей, так задавать массив нельзя.

Так неправильно:

In [29]:
np.zeros(5, 7)

TypeError: ignored

### `Способы создания NumPy array`

5. На основе списков

In [30]:
print(np.array([[1, 2, 3], [4, 5, 6]]))

[[1 2 3]
 [4 5 6]]


Число элементов должно удовлетворять размерности, numpy попытается преобразовать типы:

In [31]:
print(np.array([[1, 2, 3], [4, 6]]))

[list([1, 2, 3]) list([4, 6])]


Тип можно указать явно, и неправильное число элементов приведёт к ошибке:

In [32]:
np.array([[1, 2, 3], [4, 6]], dtype=np.float32)

ValueError: ignored

### `Способы создания NumPy array`

6. Последовательность целых чисел заданной длины

In [33]:
# Синтаксис аналогичен функции `range`, но в этом случае данные хранятся в памяти
np.arange(2, 10)

array([2, 3, 4, 5, 6, 7, 8, 9])

7. Разбиение отрезка на равные подотрезки (в исходном и log-масштабе)

In [34]:
np.linspace(0.0, 1.0, 5), np.logspace(0.0, 1.0, 5)

(array([0.  , 0.25, 0.5 , 0.75, 1.  ]),
 array([ 1.        ,  1.77827941,  3.16227766,  5.62341325, 10.        ]))

### `Способы создания NumPy array`

<font color='brown'>**Задача 2.** Задайте массив `result` размером $50$ на $30$, состоящий из троек:</font>

In [156]:
### your code here

<font color='brown'>**Задача 3.** Какая размерность будет у массива, полученного с помощью команды `np.array([[1], [2]])`?</font>

In [5]:
### your answer here

### `Важные параметры создания ndarray`

- `shape` — список или кортеж с размерностями создаваемого массива<br><br>

- `dtype` — указание типа элементов массива, если массив создаётся на базе объектов из Python, то должно существовать преобразование из типа этих объектов в указываемый тип<br><br>

- `order` — порядок хранения данных в памяти, по строкам (C-order) или же по столбцам (Fortran-order). По-умолчанию используется C-order

### `Отличие классов np.ndarray и np.matrix`

- `ndarray` — более общий класс, поддерживающий все возможные операции<br><br>

- `matrix` — более узкий класс, наследующий `ndarray`. Он поддерживает несколько операций и атрибутов, специфичных для матриц, в удобной нотации<br><br>

- При этом все те же операции можно применять и к `ndarray`, если он двумерный и состоит из чисел (начиная с Python 3.5 умножение можно записать как `A @ B`)<br><br>

- Рекомендуется пользоваться `ndarray`, чтобы не вносить путаницу в код и не проверять каждый раз, какого именно типа массив будет обрабатываться

### `Изменение размерности`

`reshape` позволяет изменить размерности массива __без изменения общего числа элементов__:

In [44]:
a = np.zeros([2, 3, 2])
print(a, a.shape)

[[[0. 0.]
  [0. 0.]
  [0. 0.]]

 [[0. 0.]
  [0. 0.]
  [0. 0.]]] (2, 3, 2)


In [4]:
b = a.reshape((1, 2 * 3 * 2))
print(b, b.shape, '\n')

b[0][0] = 10  # 'b' is a new view on the same data

print(a, a.shape)

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]] (1, 12) 

[[[10.  0.]
  [ 0.  0.]
  [ 0.  0.]]

 [[ 0.  0.]
  [ 0.  0.]
  [ 0.  0.]]] (2, 3, 2)


Можно проверить, не пересекаются ли в памяти два массива:

In [54]:
np.may_share_memory(a, b)

True

### `Изменение размерности`

Отметим, что практически все методы `np.ndarray` доступны в виде функций модуля NumPy:

In [ ]:
c = np.reshape(b, (2 * 3, 2))
print(c, c.shape)

[[10.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]] (6, 2)


Размерность можно изменить и напрямую:

In [ ]:
c.shape = (1, 2 * 3 * 2)
print(c, c.shape)

[[10.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]] (1, 12)


Можно "не заполнять" одну размерность итогового массива, она заполнится автоматически:

In [ ]:
d = c.reshape(2, -1, 3)
print(d, d.shape)

[[[10.  0.  0.]
  [ 0.  0.  0.]]

 [[ 0.  0.  0.]
  [ 0.  0.  0.]]] (2, 2, 3)


### `Про размерности массивов`

In [ ]:
a = np.array([1, 2, 3])
b = np.array([[1, 2, 3]])
c = np.array([[1], [2], [3]])
print(a.shape)
print(b.shape)
print(c.shape)

(3,)
(1, 3)
(3, 1)


- Данные лежат в памяти одним и тем же образом

- Вопрос только во `view` $==$ способ индексации $==$ число и порядок индексов

In [ ]:
print(a.ndim, b.ndim, c.ndim)

1 2 2


Полезная для понимания происходящего статья: https://stackoverflow.com/questions/22053050/difference-between-numpy-array-shape-r-1-and-r

### `View и копирование`

View ссылается на те же данные, но позволяет задать другие размерности массива.

In [18]:
x = np.arange(10)
v = x.view()
v.shape = (2, 5)
x, v

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([[0, 1, 2, 3, 4],
        [5, 6, 7, 8, 9]]))

In [20]:
v[0, 0] = 100
x, v

(array([100,   1,   2,   3,   4,   5,   6,   7,   8,   9]),
 array([[100,   1,   2,   3,   4],
        [  5,   6,   7,   8,   9]]))

Если нужно получить копию массива, чтобы не портить переданные данные, пользуйтесь методом `copy`:

In [21]:
x = np.arange(10)
y = x.copy()
y[:] = 0
x, y

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]))

### `Индексирование`

- Всё аналогично спискам, можно делать срезы и использовать отрицательные индексы
- Индексы и срезы в многомерных массивах не обязательно разделять квадратными скобками
- Подробное описание: https://docs.scipy.org/doc/numpy/reference/arrays.indexing.html

In [ ]:
a = np.array([[1, 2, 3], [3, 4, 5]])

print(a, '\n')
print(a[0], '\n')
print(a[0, 0: 100500], '\n')
print(a[0][1], '\n')
print(a[-1][-2], '\n')
print(a[-1, -2], '\n')
print(a[0, 0: -1])

[[1 2 3]
 [3 4 5]] 

[1 2 3] 

[1 2 3] 

2 

4 

4 

[1 2]


### `Индексирование`

Важное отличие от питоновских списков: при slicing возвращается **view**, а не копия! Это позволяет присваивать значения подматрицам.

In [69]:
a = np.array([[1, 2, 3], [3, 4, 5]])
print(a, '\n')

a[:, 1] = 10
print(a, '\n')

[[1 2 3]
 [3 4 5]] 

[[ 1 10  3]
 [ 3 10  5]] 



### `Логическое индексирование`

Индексирование можно производить по логическому массиву такого же размера:

In [8]:
a = np.arange(10)
i = np.array([j % 2 == 0 for j in range(10)])

print(a, '\n')
print(i, '\n')
print(a[i], '\n')

[0 1 2 3 4 5 6 7 8 9] 

[ True False  True False  True False  True False  True False] 

[0 2 4 6 8] 



При этом такие слайсы также позволяют менять исходный массив:

In [ ]:
a[i] = 11
a

array([11,  1, 11,  3, 11,  5, 11,  7, 11,  9])

### `Логическое индексирование`

С помощью функции `where`, можно находить индексы элементов, заданные маской:

In [11]:
np.where(a > 4)

(array([0, 5, 6, 7, 8, 9]),)

В случае одного аргумента `np.where` вернет `tuple`. Смотри документацию или [тут](https://stackoverflow.com/questions/50646102/what-is-the-purpose-of-numpy-where-returning-a-tuple)

In [6]:
c = np.arange(12).reshape(3, 2, 2)
np.where(c > 4)

[[[ 0  1]
  [ 2  3]]

 [[ 4  5]
  [ 6  7]]

 [[ 8  9]
  [10 11]]]


(array([1, 1, 1, 2, 2, 2, 2]),
 array([0, 1, 1, 0, 0, 1, 1]),
 array([1, 0, 1, 0, 1, 0, 1]))

In [164]:
c

array([[[ 0,  1],
        [ 2,  3]],

       [[ 4,  5],
        [ 6,  7]],

       [[ 8,  9],
        [10, 11]]])

In [103]:
np.where(a > 4, a, 0)

array([0, 0, 0, 0, 0, 5, 6, 7, 8, 9])

<font color='brown'> **Задача 4.** Даны два вектора одинаковой длины: `a` и `b`. Оставить в этих векторах только те элементы, которые соответствуют позициям ненулевых элементов в обоих векторах. </font>

In [15]:
a = np.array([0, 1, 0, 2, 3])
b = np.array([1, 5, 2, 0, 6])

In [17]:
### your code here


Проверьте себя:

In [26]:
assert a.tolist() == [1, 3]
assert b.tolist() == [5, 6]

### `Сложное индексирование`

По каждой размерности подаются массивы одинаковых размеров, элементы которых соответствуют индексам каждой размерности. Тогда, на выходе будет массив размера этих массивов, элементы которого будут соответствовать элементам исходного массива, взятых в точках, соответствующих поданным массивам.

In [18]:
X = np.array([[1, 2, 3], [4, 5, 6]])
X

array([[1, 2, 3],
       [4, 5, 6]])

In [19]:
X[[0, 1], [1, 2]], X[[0, 0], [1, 0]]

(array([2, 6]), array([2, 1]))

In [ ]:
Y = np.arange(24).reshape((3, 2, 4))
Y

array([[[ 0,  1,  2,  3],
        [ 4,  5,  6,  7]],

       [[ 8,  9, 10, 11],
        [12, 13, 14, 15]],

       [[16, 17, 18, 19],
        [20, 21, 22, 23]]])

In [ ]:
Y[[1, 0], :, [2, 1]]

array([[10, 14],
       [ 1,  5]])

### `Сокращенная индексация`

Если у вас есть многомерный массив, а вам, например, надо взять определенные индексы только с последней размерности, то можно написать так:

In [ ]:
x = np.arange(80).reshape((2, 2, 4, 5))

print(x[..., 0])
x[..., 0].shape

[[[ 0  5 10 15]
  [20 25 30 35]]

 [[40 45 50 55]
  [60 65 70 75]]]


(2, 2, 4)

Более хитрый пример:

In [ ]:
x = np.arange(720).reshape((3, 4, 5, 3, 2, 2))

print(x[2, :, 3, ..., 0])
x[2, :, 3, ..., 0].shape

[[[516 518]
  [520 522]
  [524 526]]

 [[576 578]
  [580 582]
  [584 586]]

 [[636 638]
  [640 642]
  [644 646]]

 [[696 698]
  [700 702]
  [704 706]]]


(4, 3, 2)

### `Арифметические операции`

- Арифметические операции в общем случае по-элементные и требуют одинакового размера операндов<br>

- Но часто NumPy может применять их к операндам разного размера с помощью broadcasting, то есть правил обработки операндов разного размера<br><br>

Примеры операций с массивами одного размера:

In [29]:
A, B = np.array(A), np.array(B)  # were declared previously

print(A, '\n\n', B, '\n\n', 2 * A, '\n\n', A + B)

[[1 2 3]
 [4 5 6]
 [7 8 9]] 

 [[1 0 0]
 [0 2 0]
 [0 0 3]] 

 [[ 2  4  6]
 [ 8 10 12]
 [14 16 18]] 

 [[ 2  2  3]
 [ 4  7  6]
 [ 7  8 12]]


### `Поэлементные арифметические операции`

In [30]:
np.sin(A)

array([[ 0.84147098,  0.90929743,  0.14112001],
       [-0.7568025 , -0.95892427, -0.2794155 ],
       [ 0.6569866 ,  0.98935825,  0.41211849]])

In [31]:
np.abs(A)

array([[1, 2, 3],
       [4, 5, 6],
       [7, 8, 9]])

In [32]:
A ** 0.5

array([[1.        , 1.41421356, 1.73205081],
       [2.        , 2.23606798, 2.44948974],
       [2.64575131, 2.82842712, 3.        ]])

Поэлементые операции сравнения:

In [33]:
A > 3

array([[False, False, False],
       [ True,  True,  True],
       [ True,  True,  True]])

### `Матричные арифметические операции`

In [34]:
print(A - B)

[[0 2 3]
 [4 3 6]
 [7 8 6]]


In [35]:
print(A * B)

[[ 1  0  0]
 [ 0 10  0]
 [ 0  0 27]]


In [36]:
print(B / A)

[[1.         0.         0.        ]
 [0.         0.4        0.        ]
 [0.         0.         0.33333333]]


In [37]:
print(A @ B)

[[ 1  4  9]
 [ 4 10 18]
 [ 7 16 27]]


### `Фиктивная размерность`

Два способа добавить фиктивную размерность в произвольный массив:

In [ ]:
d[:, np.newaxis].shape

(2, 1, 2, 3)

In [ ]:
d[None, :].shape

(1, 2, 2, 3)

Для одномерного массива можно использовать `reshape`:

In [ ]:
x = np.arange(7)

x, x.reshape(-1, 1)

(array([0, 1, 2, 3, 4, 5, 6]),
 array([[0],
        [1],
        [2],
        [3],
        [4],
        [5],
        [6]]))

Зачем нужно добавлять фиктивную размерность? Например, чтобы проводить различные операции с массивами неравного размера.

### `Broadcasting`

Подробное описание: http://docs.scipy.org/doc/numpy/user/basics.broadcasting.html

Пусть нам дана матрица $X$ размером $5 \times 5$ и вектор $y$ длины $5$. Пусть мы хотим прибавить вектор к каждой строке матрицы.

In [ ]:
x = np.arange(25).reshape(5, 5)
y = np.arange(5)

x, y

(array([[ 0,  1,  2,  3,  4],
        [ 5,  6,  7,  8,  9],
        [10, 11, 12, 13, 14],
        [15, 16, 17, 18, 19],
        [20, 21, 22, 23, 24]]),
 array([0, 1, 2, 3, 4]))

Наивный способ решения проблемы будет работать правильно!

In [ ]:
x + y

array([[ 0,  2,  4,  6,  8],
       [ 5,  7,  9, 11, 13],
       [10, 12, 14, 16, 18],
       [15, 17, 19, 21, 23],
       [20, 22, 24, 26, 28]])

**Почему?** Правила broadcasting (приведение размеров).

1. Если два массива имеют размерности $(a_1, a_2, ..., a_n)$ и $(b_1, b_2, ..., b_n)$ соответственно, то между ними можно проводить поэлементные операции, если для каждого $i$ выполнено одно из трёх условий:
    * $a_i = b_i$
    * $a_i = 1$
    * $b_i = 1$
    
2. Если поэлементная операция выполняется между массивами разного размера, то к массиву меньшего размера добавляются ведущие фиктивные размерности.

### `Арифметика и broadcasting`

Если по одному из измерений массивы не равны, и у одного из них эта размерность имеет длину 1, то он будет продублирован по этой размерности:

In [ ]:
a = np.array([[1, 2], [3, 4]])
b = np.array([1])
c = np.array([1, 1])

print(b + c, '\n')
print(a + b, '\n')
print(a + c)

[2 2] 

[[2 3]
 [4 5]] 

[[2 3]
 [4 5]]


### `Арифметика и broadcasting`

Если не совпадает количество размерностей, то массив, у которого их меньше, будет "добиваться" слева размерностями длины 1:

In [ ]:
a = np.ones((2, 3, 4))
b = np.ones(4)

print(a + b) # here a.shape=(2, 3, 4) and b.shape is considered to be (1, 1, 4)

[[[2. 2. 2. 2.]
  [2. 2. 2. 2.]
  [2. 2. 2. 2.]]

 [[2. 2. 2. 2.]
  [2. 2. 2. 2.]
  [2. 2. 2. 2.]]]


### `Арифметика и broadcasting`

Добавим к массиву вектор-строку:

In [ ]:
a = np.array([[1, 2, 3], [4, 5, 6]])
b = np.array([1, 2, 3])
a + b

array([[2, 4, 6],
       [5, 7, 9]])

Теперь попробуем добавить столбец:

In [40]:
a = np.array([[1, 2, 3], [4, 5, 6]])
b = np.array([1, 2])
a + b

ValueError: ignored

### `Арифметика и broadcasting`

- Ошибка возникла из-за того, что прибавлялась строка с неправильным размером<br>

- Нужно преобразовать строку в столбец (добавить фиктивную размерность), и тогда NumPy поймёт, как с ней работать<br><br>

Для этого воспользуемся `reshape` (способ хороший, но далеко не единственный):

In [41]:
a + np.reshape(b, (2, 1))

array([[2, 3, 4],
       [6, 7, 8]])

### `Арифметика и broadcasting`

**Всегда проверяйте операции с броадкастингом! Легко можно напороться на неприятности**

<font color='brown'>**Задача 5.** Какие из этих команд будут выполняться с ошибкой?</font>

1. `np.ones((2, 3)) + np.ones(3)`

2. `np.ones(2) + np.ones((2, 3))`

3. `np.zeros((4, 3)) + np.ones((4, 1))`

4. `np.zeros((3, 4)) + np.ones((4, 3))`

5. `np.zeros((1, 3, 5)) + np.zeros((1, 3))`

6. `np.zeros((5, 3, 1)) + np.zeros((1, 5))`

In [48]:
### your answer here

<font color='brown'>**Задача 6.** Пусть нам дана матрица $X$ размером $10 \times 10$ и вектор-столбец $y$ длины $10$. Получите матрицу `result`, полученную прибавлением к каждому столбцу $X$ вектора $y$ (без использования циклов). </font>

In [50]:
x = np.arange(100).reshape(10, 10)
y = np.arange(10)

In [58]:
## your code here

Проверьте себя:

In [57]:
assert result[0][0] == 0
for i in range(1, 10):
    assert result[i][i-1] == 10 * i + (i - 1) * 2 + 1

### `Матричное умножение`

Расмотрим для случая двумерных матриц, кому интересны многомерные, изучайте
https://numpy.org/devdocs/reference/generated/numpy.dot.html

In [59]:
print(A @ B, '\n')
print(A.dot(B), '\n')
print(np.dot(A, B))

[[ 1  4  9]
 [ 4 10 18]
 [ 7 16 27]] 

[[ 1  4  9]
 [ 4 10 18]
 [ 7 16 27]] 

[[ 1  4  9]
 [ 4 10 18]
 [ 7 16 27]]


### `Транспонирование`

- Расмотрим для случая двумерных матриц, кому интересны многомерные, изучайте
https://stackoverflow.com/questions/32034237/how-does-numpys-transpose-method-permute-the-axes-of-an-array
- При транспонировании (как и `reshape`) возвращается ссылка на те же данные

In [ ]:
print(A, '\n\n', A.T, '\n\n', A.transpose())

[[1 2 3]
 [4 5 6]
 [7 8 9]] 

 [[1 4 7]
 [2 5 8]
 [3 6 9]] 

 [[1 4 7]
 [2 5 8]
 [3 6 9]]


Стоит быть осторожным с транспонированием:

In [ ]:
# in memory: 0, 1, 2, 3
a = np.arange(4).reshape(2, 2)
b = a.reshape(-1)
c = a.T.reshape(-1)
np.may_share_memory(a, b), np.may_share_memory(a, c)

(True, False)

In [ ]:
d = a.T.view()
np.may_share_memory(a, d)
d.flags['C_CONTIGUOUS']

False

In [ ]:
print(a.flags['C_CONTIGUOUS'])
print(a.T.flags['C_CONTIGUOUS'])

True
False


### `Сравнение скорости`

Воспользуемся декоратором `timed` для сравнения скорости матричных операций, реализованных в `numpy` и написанных выше с помощью списков:

In [62]:
@timed
def matrix_add_np(A, B): return A + B

@timed
def matrix_mul_np(A, B): return A @ B

In [67]:
tmp = [range(1000) for _ in range(1000)]
X, Y = np.array(tmp), np.array(tmp)

_ = matrix_add(X, Y)
_ = matrix_add_np(X, Y)

tmp = [range(200) for _ in range(200)]
X, Y = np.array(tmp), np.array(tmp)

_ = matrix_mul(X, Y)
_ = matrix_mul_np(X, Y)

matrix_add  690.307 ms

matrix_add_np  3.761 ms

matrix_transpose  12.119 ms

matrix_mul  2138.916 ms

matrix_mul_np  8.289 ms



### `Агрегирующие функции`

Подобных функций много, ищите нужное в документации NumPy

In [68]:
a = np.array([1, 2, 3, 4, 3, 2, 1, 3, 3])

print('Min element:          {}'.format(np.min(a)))
print('Min element position: {}'.format(np.argmin(a)))
print('Max element:          {}'.format(np.max(a)))
print('Mean:                 {}'.format(np.mean(a)))
print('Sum:                  {}'.format(np.sum(a)))
print('Median:               {}'.format(np.median(a)))
print('Cumulative sum:       {}'.format(np.cumsum(a)))
print('Cumulative product:   {}'.format(np.cumprod(a)))

Min element:          1
Min element position: 0
Max element:          4
Mean:                 2.4444444444444446
Sum:                  22
Median:               3.0
Cumulative sum:       [ 1  3  6 10 13 15 16 19 22]
Cumulative product:   [   1    2    6   24   72  144  144  432 1296]


Что будет происходить в случае многомерного массива?

### `Агрегирующие функции`

В многомерном случае операция применяется к массиву, вытянутому в вектор (flatten):

In [69]:
a = np.array([[1, 2, 3], [4, 3, 2], [1, 3, 3]])
print(a, '\n')
print(np.max(a))
print(np.cumsum(a))

[[1 2 3]
 [4 3 2]
 [1 3 3]] 

4
[ 1  3  6 10 13 15 16 19 22]


Для работы с определёнными размерностям нужно указать явно параметр `axis`:

In [70]:
print(np.max(a, axis=0), '\n')
print(np.cumsum(a, axis=1))

[4 3 3] 

[[1 3 6]
 [4 7 9]
 [1 4 7]]


Как легко запомнить, как работает axis:
* после применения функции получится массив, в котором будет отсутствовать размерность указанная в `axis`, но остальные размерности будут иметь такие же значения

### `Агрегирующие функции`

<font color='brown'>**Задача 7.** Дан вектор $x$ и квадратная матрица $D$. Вычислить вектор значений $y_j = argmin_i (x_i + D_{ij})$. </font>

In [72]:
x = np.array([5, 2, 3, 1])
D = np.array([
    [1, 2, 3],
    [2, 2, 4],
    [5, 6, 1],
    [2, 4, 5]
])

In [74]:
### your code here

Проверьте себя:

In [ ]:
assert y.tolist() == [3, 1, 2]

### `Конкатенация массивов`

Конкатенировать несколько массивом можно с помощью функций `concatenate` (общий случай), `hstack` и `vstack`:

In [75]:
np.hstack([A, B])

array([[1, 2, 3, 1, 0, 0],
       [4, 5, 6, 0, 2, 0],
       [7, 8, 9, 0, 0, 3]])

In [77]:
np.vstack([A, B])

array([[1, 2, 3],
       [4, 5, 6],
       [7, 8, 9],
       [1, 0, 0],
       [0, 2, 0],
       [0, 0, 3]])

### `Ещё несколько полезных функций`

Чтобы построить двумерный массив, состоящий из повторений одномерных, можно использовать `np.tile`:

In [78]:
a = np.array([2, 3, 4])
np.tile(a[np.newaxis, :], (2, 1))

array([[2, 3, 4],
       [2, 3, 4]])

Для того, чтобы повторить каждый элемент в массиве, можно использовать `np.repeat`

In [79]:
np.repeat([1, 2, 3], repeats=2, axis=0)

array([1, 1, 2, 2, 3, 3])

In [80]:
print(np.all(A == A))
print(np.all(A == B))
print(np.all(A == [1, 2, 3], axis=1))

True
False
[ True False False]


In [81]:
func = np.vectorize(lambda x: x ** 2)
func(A)

array([[ 1,  4,  9],
       [16, 25, 36],
       [49, 64, 81]])

### `Типы и преобразование типов`

Атрибут `dtype` хранит информацию о типе массива:

In [82]:
x = np.arange(15)
y = np.array([1.5, 2.5])

x.dtype, y.dtype

(dtype('int64'), dtype('float64'))

Преобразование типов осуществляется с помощью метода `astype`:

In [83]:
x = x.astype(np.float64)
x.dtype

dtype('float64')

Можно сразу указывать желаемый тип при создании массивов при помощи параметра `dtype`:

In [84]:
np.ones((2, 3), dtype=str)

array([['1', '1', '1'],
       ['1', '1', '1']], dtype='<U1')

### `Примеры задач на матрицы в NumPy`

Условие - нельзя использовать
- циклы
- генераторы списков/списковые включения
- map-функции (в т.ч. `np.vectorize`)

<font color='brown'> **Задача 8**. Пронумеровать в порядке следования максимальные элементы в векторе. Для вектора `[1, 2, 3, 3, 2, 1, 3, 1]` должно получиться `[0, 0, 1, 2, 0, 0, 3, 0]`: </font>

In [ ]:
def task_1(a):
    pass
    ### your code here

print(task_1([1, 2, 3, 3, 2, 1, 3, 1]))

[0 0 1 2 0 0 3 0]


### `Примеры задач на матрицы в NumPy`

<font color='brown'> **Задача 9**. Реализуйте функцию подсчёта произведения ненулевых элементов на диагонали прямоугольной матрицы. Для матрицы `[[1, 0, 1], [2, 0, 2], [3, 0, 3], [4, 4, 4]]` ответом является $3$. Если ненулевых элементов нет, функция должна возвращать `None`: </font>

In [ ]:
def task_2(A):
    pass
    ### your code here

print(task_2(np.array([[1, 0, 1], [2, 0, 2], [3, 0, 3], [4, 4, 4]])))

3


<font color='brown'> **Задача 10**. Найти максимальный элемент в вектор-строке среди элементов, перед которым стоит нулевой. Для `[0, 4, 2, 0, 3, 0, 0, 5, 7, 0]` ответ равен $5$: </font>

In [ ]:
def task_3(a):
    pass
    ### your code here

print(task_3(np.array([0, 4, 2, 0, 3, 0, 0, 5, 7, 0])))

5


## `Спасибо за внимание!`